In [2]:
import os
import random
import shutil, os
import pandas as pd

from os import listdir
from os.path import isfile, join

In [3]:
# import df
# take out 100 images for fisher, squirrels, grouse and copy to snapshot_wisconsin_expert_resized_yolo_test


In [7]:
expert_label_df = pd.read_excel('Berkeley-sswi_metadata (1).xlsx')

In [9]:
#Percent mulitple animal classes

(len(expert_label_df.TRIGGER_ID) - expert_label_df.TRIGGER_ID.nunique())/(len(expert_label_df.TRIGGER_ID))

0.010578857185427996

In [18]:
sample_species = ['Fisher', 'Grouse, Ruffed', 'Squirrels and Chipmunks']

i = 0
for species in sample_species:
    expert_label_df_sample_int = expert_label_df[expert_label_df['CLASS_SPECIES'] == species].sample(n=100, random_state=1)
    
    if i ==0:
        expert_label_df_sample = expert_label_df_sample_int
        
    else:
        expert_label_df_sample = pd.concat([expert_label_df_sample, expert_label_df_sample_int])
        
    i += 1

In [31]:

for trigger, rows in expert_label_df_sample.iterrows():
    trigger_id = rows['TRIGGER_ID']
    
    for image in ['A', 'B', 'C']:
        try:
            shutil.copy('snapshot_wisconsin_expert_resized/' + trigger_id + image + '.jpg',
                        'snapshot_wisconsin_expert_resized_yolo_test')
        except:
            continue

In [9]:
# move 1500 wolf images to train

wolf_random = expert_label_df[expert_label_df['CLASS_SPECIES'] == 'Wolf'].sample(frac=1)['TRIGGER_ID']
    

In [13]:
batch = 150


for i in range(1,4):
    wolf_to_copy_list = list(wolf_random)[(i*batch - batch): (i*batch)]
    
    for event in wolf_to_copy_list:
        for image in ['A', 'B', 'C']:
            try:
                shutil.copy('snapshot_wisconsin_expert_resized/' + event + image + '.jpg',
                            'WOLF(EXPERT)/WOLF' + str(i + 1))
            except:
                continue

In [ ]:
# Adding additional deer, elk and coyote images to test set to give larger test size

In [14]:
used_triggers = []

splits = ['train/', 'val/', 'test/']

for split in splits:
    for im in os.listdir('/Users/sleung2/Documents/MIDS Program/Capstone_local/snapshot_wisconsin/all/yolo_splits4/' + split + 'images/'):
        used_triggers.append(im)
        
        

In [22]:
used_triggers_SW = [im for im in used_triggers if 'SSWI' in im]

used_trig_SW = [im[:-5] for im in used_triggers_SW]


used_trig_SW_set = list(set(used_trig_SW))

In [29]:
unused_trigger_df = expert_label_df[~expert_label_df['TRIGGER_ID'].isin(used_trig_SW_set)]

In [32]:
species_remove = ['Fisher',
                 'Skunk, Striped',
                 'Grouse, Ruffed',
                 'Otter',
                 'Mink',
                 'Weasel',
                 'Porcupine',
                 'Squirrels and Chipmunks']

unused_trigger_df = unused_trigger_df[~unused_trigger_df['CLASS_SPECIES'].isin(species_remove)]

In [34]:
unused_trigger_df.CLASS_SPECIES.unique()

array(['Coyote', 'Deer', 'Cottontail', 'Fox, Red', 'Wolf', 'Elk',
       'Raccoon', 'Snowshoe Hare', 'Opossum', 'Turkey', 'Fox, Gray',
       'Bear'], dtype=object)

In [36]:
unused_trigger_df.groupby('CLASS_SPECIES').count()

,TRIGGER_ID,Total
CLASS_SPECIES,,
Bear,6,6
Cottontail,19,19
Coyote,297,297
Deer,3704,3704
Elk,1856,1856
"Fox, Gray",2,2
"Fox, Red",76,76
Opossum,16,16
Raccoon,16,16


In [37]:
unused_trigger_dce_df = unused_trigger_df[unused_trigger_df['CLASS_SPECIES'].isin(['Deer', 'Coyote', 'Elk'])]

In [39]:
unused_trigger_dce_sample = unused_trigger_dce_df.sample(n=250)

In [41]:
unused_trigger_dce_sample.groupby('CLASS_SPECIES').count()

,TRIGGER_ID,Total
CLASS_SPECIES,,
Coyote,15,15
Deer,161,161
Elk,74,74


In [44]:
top_path = '/Users/sleung2/Documents/MIDS Program/Capstone_local/snapshot_wisconsin/all/'

for row, values in unused_trigger_dce_sample.iterrows():
    event = values['TRIGGER_ID']
    
    for letter in ['A', 'B', 'C']:
        img_file = '/Users/sleung2/Documents/MIDS Program/Capstone_local/snapshot_wisconsin_expert_resized/' + event + letter + '.jpg'
            
        try:
            shutil.copy(img_file,
                     top_path + 'yolo_splits4/test/images/' +  event + letter + '.jpg')
         

        except:
            print(event)
            

In [45]:
#NEW LIST of test images

new_test_triggers = []

splits = ['test/']

for split in splits:
    for im in os.listdir('/Users/sleung2/Documents/MIDS Program/Capstone_local/snapshot_wisconsin/all/yolo_splits4/' + split + 'images/'):
        new_test_triggers.append(im)

In [47]:


new_trig_SW = [im[:-5] for im in new_test_triggers]


new_trig_SW_set = list(set(new_trig_SW))

In [55]:
expert_label_used_df = expert_label_df[expert_label_df['TRIGGER_ID'].isin(new_trig_SW_set)]

In [53]:
species_dist_df = pd.read_excel('species_distribution_test.xlsx')




In [61]:
test_labels = pd.merge(expert_label_used_df, species_dist_df[['CLASS_SPECIES','CLASS_SPECIES_RESTATED']],
        how = 'left',
        on = 'CLASS_SPECIES')#.to_csv('test_labels.csv')

In [63]:
test_dist = test_labels.groupby('CLASS_SPECIES_RESTATED').count().reset_index()

In [65]:
test_dist['perc'] = test_dist['CLASS_SPECIES']/test_dist['CLASS_SPECIES'].sum()

In [67]:
test_dist.sort_values(by = 'perc')

,CLASS_SPECIES_RESTATED,TRIGGER_ID,CLASS_SPECIES,Total,perc
6,opossum,52,52,52,0.035254
5,foxgray_foxred,70,70,70,0.047458
9,turkey,74,74,74,0.050169
7,other,83,83,83,0.056271
8,raccoon,119,119,119,0.080678
1,cottontail_snowshoehare,122,122,122,0.082712
10,wolf,126,126,126,0.085424
0,bear,130,130,130,0.088136
2,coyote,150,150,150,0.101695
4,elk,197,197,197,0.133559
